In [1]:
import sys
sys.path.append(r'C:\Users\Amr\Desktop\Ai\object_detection\Project_1_car_counter\sort.py')

In [3]:
from ultralytics import YOLO
import cv2

In [4]:
from sort import *

In [19]:
model = YOLO('Desktop//Ai//object_detection//YOLO//yolov8s.pt')


In [20]:
results = model(r"C:\Users\Amr\Desktop\Ai\object_detection\pictures\1.jpg", show=False)


image 1/1 C:\Users\Amr\Desktop\Ai\object_detection\pictures\1.jpg: 448x640 2 persons, 1 bus, 1 truck, 1 backpack, 89.6ms
Speed: 2.0ms preprocess, 89.6ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)


In [21]:
# import matplotlib.pyplot as plt

# processed_image = results[0].plot()  # >> return numpy array for the image making it proper for imshow
# processed_image = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)

# plt.figure(figsize=(12, 8))  
# plt.imshow(processed_image)
# plt.axis("off")
# plt.show()


In [22]:
import cvzone

In [23]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

In [24]:
detections = np.empty((0,5))

In [27]:
# for webcam :
# cap = cv2.VideoCapture(0)
# cap.set(3, 1280)
# cap.set(4, 720)

# for videos 
cap = cv2.VideoCapture(r'C:\Users\Amr\Desktop\Ai\object_detection\videos\cars.mp4')
mask = cv2.imread(r"C:\Users\Amr\Desktop\Ai\object_detection\pictures\region.png")



# tracking : assigning objects from frame to another
tracker = Sort(max_age = 20 , min_hits= 3, iou_threshold= 0.15)
limits = [380, 297, 673, 297]
totalCount = []

while True :
    ret, frame = cap.read()
    imgRegion = cv2.bitwise_and(frame, mask) # used for specify the region we want to detect
    results = model(imgRegion) 
        
    detections = np.empty((0,5)) # used for the tracker to know the detections and append on them 

    
    for box in results[0].boxes :
        # bounding box
        x1,y1,x2,y2 = box.xyxy[0]
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        # cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,255),3)
        w,h = x2-x1 , y2-y1 
        #  cvzone.cornerRect(frame,[x1,y1,w,h],l=5)

        #confidence        
        conf = box.conf.item()  # Convert tensor to float
        conf = round(conf,2)*100

        # class name
        cls = int(box.cls[0])
        # cvzone.putTextRect(frame, f'{classNames[cls]} {conf}', (max(0,x1),max(30, y1)),\
        # scale=1, thickness=1, offset=3)
            
        #classes wanted 
        if classNames[cls] in ['car','motorbike','bus','truck'] and conf >=15:
                # cvzone.putTextRect(frame, f'{classNames[cls]} {conf}', (max(0,x1),max(30, y1)),\
                # scale=1, thickness=1, offset=3)
                 
                # cvzone.cornerRect(frame,[x1,y1,w,h],l= 9, rt= 5)
                cvzone.cornerRect(frame,[x1,y1,w,h],l= 9, rt= 2, colorR=(255,0,255))
                cvzone.putTextRect(frame, f'{(classNames[cls])}', (max(0,x1),max(30, y1)),\
                scale=2, thickness=1, offset=6)
                
                # for tracker                
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack([detections, currentArray])
                
        
                
                





    # tracker     
    tracker_results = tracker.update(detections)
    cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
    for result in tracker_results :
            x1, y1, x2, y2, id = result
            x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
            print(result)
            cx, cy = x1 + w // 2, y1 + h // 2
            cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
            if limits[0] < cx < limits[2] and limits[1] - 20 < cy < limits[1] + 20:
                if totalCount.count(id) == 0:
                        totalCount.append(id)
                        cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)
            
            
        #     cvzone.cornerRect(frame,[x1,y1,w,h],l= 9, rt= 2, colorR=(255,0,255))
        #     cvzone.putTextRect(frame, f'{int(id)}', (max(0,x1),max(30, y1)),\
        #     scale=1, thickness=1, offset=6)
            
            
    cvzone.putTextRect(frame, f' Count: {len(totalCount)}', (50, 50))           
    cv2.imshow("Webcam Feed", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break    
    
cap.release()   
cv2.destroyAllWindows()

0: 384x640 5 cars, 1 truck, 97.6ms
Speed: 9.8ms preprocess, 97.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[        570         318         651         424         248]
[        536         179         557         196         247]
[        570         315         651         426         246]
[        626         182         656         206         245]
[        378         348         496         455         244]
[        458         228         509         271         243]

0: 384x640 5 cars, 1 truck, 94.0ms
Speed: 2.4ms preprocess, 94.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[     567.75       322.8      648.25       435.2         248]
[     534.97      179.02      557.03      196.98         247]
[     567.69      321.44      650.31      436.56         246]
[     625.59      181.36      655.41      206.64         245]
[     374.25      350.78      492.75      457.22         244]
[     456.12       229.9      507.88       273.1     